In [1]:
import torch

In [2]:
from torch import nn
from torch.autograd import Variable

In [3]:
import numpy as np
N_train = 5000
# Load Data # replace with something else (also will make .pkl file)
data = np.load('single_locus_data.npy', allow_pickle='TRUE')
# with open("single_locus_data.pkl", "wb") as tf:
#     data = pickle.load(tf)    
f = data.item()['f'][:, None]
t = data.item()['t'][:, None]
p = data.item()['phi'][:, None]
print(len(f))
idx_list = np.arange(len(f))
np.random.shuffle(idx_list)
print(idx_list)

f_train = torch.from_numpy(f[idx_list[:N_train]]).float()
t_train = torch.from_numpy(t[idx_list[:N_train]]).float()
p_train = torch.from_numpy(p[idx_list[:N_train]]).float()
f_test = torch.from_numpy(f[idx_list[N_train:]]).float()
t_test = torch.from_numpy(t[idx_list[N_train:]]).float()
p_test = torch.from_numpy(p[idx_list[N_train:]]).float()


11000
[7980 6947  139 ... 2881 2459 1025]


In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [7]:
class PINN(nn.Module):
    def __init__(self, layers, s0, N0):
        super().__init__()
        self.model = nn.Sequential()
        for i in range(len(layers)-3):
            self.model.add_module(f"layer{i}", nn.Linear(layers[i], layers[i+1]))
            self.model.add_module(f"activation{i}", nn.Tanh())
        self.model.add_module(f"layer{len(layers)-2}", nn.Linear(layers[-3], layers[-2]))
        self.model.add_module(f"activation{len(layers)-2}", nn.Softplus())
        self.model.add_module(f"layer{len(layers)-1}", nn.Linear(layers[-2], layers[-1]))
        self.s = Variable(torch.tensor(s0)).to(device)
        self.N = Variable(torch.tensor(N0)).to(device)

    def forward(self, f, t):
        inputs = torch.cat([f, t], 1)
        return self.model(inputs)
    def PDE(self, f, t):
        s = self.s
        N = self.N
        p = self.model(torch.cat([f, t], 1))
        p_f = torch.autograd.grad(p.sum(), f, create_graph=True)[0]
        p_t = torch.autograd.grad(p.sum(), t, create_graph=True)[0]
        p_ff = torch.autograd.grad(p_f.sum(), f, create_graph=True)[0]
        g = p_t + s * ((1 - 2*f)*p + (f - f**2)*p_f) + 1 / (2*N) * (-2*p + 2*(1 - 2*f)*p_f + (f - f**2)*p_ff)
        return g

# Input = (f, t) series, Output = p series
model = PINN(layers = [2, 20, 20, 20, 1], s0 = 0.1, N0 = 500).to(device)
        

In [41]:
print(model)

PINN(
  (model): Sequential(
    (layer0): Linear(in_features=2, out_features=20, bias=True)
    (activation0): Tanh()
    (layer1): Linear(in_features=20, out_features=20, bias=True)
    (activation1): Tanh()
    (layer3): Linear(in_features=20, out_features=20, bias=True)
    (activation3): Softplus(beta=1, threshold=20)
    (layer4): Linear(in_features=20, out_features=1, bias=True)
  )
)


In [42]:
model(f_train, t_train)

tensor([[0.1659],
        [0.1659],
        [0.1642],
        ...,
        [0.1659],
        [0.1659],
        [0.1659]], grad_fn=<AddmmBackward0>)

In [43]:
#### Now define loss based on a PDE. We need to get derivative of p against x and t.
MSE_loss = nn.MSELoss()
f_sample = np.random.uniform(low=0.0, high=1.0, size=(500,1))
pt_f_sample = Variable(torch.from_numpy(f_sample).float(), requires_grad=True).to(device)
t_sample = np.random.uniform(low=0.0, high=10000, size=(500,1))
pt_t_sample = Variable(torch.from_numpy(t_sample).float(), requires_grad=True).to(device)
loss_pde = MSE_loss(model.PDE(pt_f_sample, pt_t_sample), torch.zeros_like(model.PDE(pt_f_sample, pt_t_sample)))
# In this case, we have p_train to compare directly against estimate of p at f_train, t_train. 
loss_data = MSE_loss(model(f_train, t_train), p_train)
loss = loss_pde + loss_data

# In a different scenario, I will put in all f, t data without estimating p before training. 
# reorganize f and t series so that I have f at each time point (t_i), add - 1/n_i sum(log(p(f(t_i), t_i))) + int model(f, t_i)df
# sort the data by time point, start from t=0 till maximum t (either t_extinct or t_fix)

In [8]:
iterations = 100
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(iterations):
    optimizer.zero_grad()
    MSE_loss = nn.MSELoss()
    f_sample = np.random.uniform(low=0.0, high=1.0, size=(500,1))
    pt_f_sample = Variable(torch.from_numpy(f_sample).float(), requires_grad=True).to(device)
    t_sample = np.random.uniform(low=0.0, high=10000, size=(500,1))
    pt_t_sample = Variable(torch.from_numpy(t_sample).float(), requires_grad=True).to(device)
    loss_pde = MSE_loss(model.PDE(pt_f_sample, pt_t_sample), torch.zeros_like(model.PDE(pt_f_sample, pt_t_sample)))
    # In this case, we have p_train to compare directly against estimate of p at f_train, t_train. 
    loss_data = MSE_loss(model(f_train, t_train), p_train)
    loss = loss_pde + loss_data
    loss.backward()
    optimizer.step()

    with torch.autograd.no_grad():
        print(epoch,"training loss:", loss.data)


0 training loss: tensor(0.3438)
1 training loss: tensor(0.3079)
2 training loss: tensor(0.2741)
3 training loss: tensor(0.2418)
4 training loss: tensor(0.2101)
5 training loss: tensor(0.1777)
6 training loss: tensor(0.1528)
7 training loss: tensor(0.1326)
8 training loss: tensor(0.1147)
9 training loss: tensor(0.0990)
10 training loss: tensor(0.0872)
11 training loss: tensor(0.0777)
12 training loss: tensor(0.0700)
13 training loss: tensor(0.0638)
14 training loss: tensor(0.0591)
15 training loss: tensor(0.0557)
16 training loss: tensor(0.0536)
17 training loss: tensor(0.0524)
18 training loss: tensor(0.0521)
19 training loss: tensor(0.0525)
20 training loss: tensor(0.0533)
21 training loss: tensor(0.0545)
22 training loss: tensor(0.0557)
23 training loss: tensor(0.0569)
24 training loss: tensor(0.0579)
25 training loss: tensor(0.0586)
26 training loss: tensor(0.0590)
27 training loss: tensor(0.0591)
28 training loss: tensor(0.0588)
29 training loss: tensor(0.0582)
30 training loss: te

In [ ]:
# Return predicted s and N, plot p(f, t). Plot p(f = 1, t), which is fixation probability as a function of time.